In [28]:
import pandas as pd
import pickle
import spacy
from tqdm import tqdm
import re
from unidecode import unidecode
import html

In [2]:
df = pd.read_json("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/fin_anime_df.json")

In [3]:
# Specify the path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/popular_dict_10.pkl"

# Open the pickle file in read-binary mode
with open(pickle_file_path, "rb") as f:
    # Load the object from the pickle file
    loaded_object = pickle.load(f)

In [4]:
df2 = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/anime-dataset-2023.csv")

In [5]:
pop_list = df2[df2['anime_id'].isin(pd.DataFrame(loaded_object)['anime_id'].tolist())].sort_values('Favorites', ascending = False).head(10)['anime_id'].tolist()

In [6]:
df['popular_recs'] = [pop_list] * len(df)

In [7]:
# Specify the path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/anime_recommendations_item_knn_CF_10k_num.pkl"

# Open the pickle file in read-binary mode
with open(pickle_file_path, "rb") as f:
    # Load the object from the pickle file
    loaded_object_cf = pickle.load(f)

In [8]:
anime_anime_id_series = pd.Series(loaded_object_cf)


In [9]:
df['cf_recs'] = df['anime_anime_id'].map(anime_anime_id_series)

In [26]:
df['tokens'] = df['text'] + " " + df['Genres']  + " " + df['Source']  + " " + df['keywords'] + " " + df['Producers'] + " " + df['imdb_name_basics_primaryName'] + " " + df['anime_Synopsis'] + " " + df['Studios'] + " " + df['Name']

In [25]:
class TextPreprocessor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        tqdm.pandas()

    def lemmatize_text(self, text):
        doc = self.nlp(text)
        return [token.lemma_ for token in doc]

    def remove_stopwords(self, tokens):
        return [token for token in tokens if not self.nlp.vocab[token].is_stop]

    def remove_punctuations(self, tokens):
        return [token for token in tokens if not self.nlp.vocab[token].is_punct]

    def decode_html_entities(self, text):
        return html.unescape(text)

    def remove_html_tags(self, text):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', text)
        return cleantext

    def preprocess_dataframe(self, df, column_name):
        # Lowercase text column
        df[column_name] = df[column_name].str.lower()

        # Decode HTML entities
        df[column_name] = df[column_name].apply(self.decode_html_entities)

        # Remove HTML tags
        df[column_name] = df[column_name].apply(self.remove_html_tags)

        # Normalize Unicode characters
        df[column_name] = df[column_name].apply(unidecode)

        # Lemmatize text column
        df[column_name] = df[column_name].progress_apply(self.lemmatize_text)
        
        # Remove stopwords from text column
        df[column_name] = df[column_name].progress_apply(self.remove_stopwords)
        
        # Remove punctuations from text column
        df[column_name] = df[column_name].progress_apply(self.remove_punctuations)
        
        return df


# Create an instance of TextPreprocessor
text_preprocessor = TextPreprocessor()


In [29]:
# Preprocess the DataFrame
fin_df = text_preprocessor.preprocess_dataframe(df, 'tokens')

100%|██████████| 11565/11565 [00:01<00:00, 6704.99it/s]


In [32]:
fin_df.to_json("fin_anime_dfv1.json")